In [1]:
import matplotlib.pylab as plt, pandas as pd, MySQLdb
from matplotlib import font_manager
myFont = font_manager.FontProperties(fname='/Library/Fonts/Songti.ttc')
titleSize = 14
tipSize = 12

In [2]:
conn = MySQLdb.connect(host='localhost',user='root',passwd='',db='qyw', charset='utf8')

In [ ]:
df = pd.read_sql('''
    SELECT PROVINCE, CITY, COUNT(DISTINCT USER_ID) AS USER_CNT FROM (SELECT * FROM (SELECT * FROM qyw.qyw_4th_visit WHERE USER_ID > 0 AND HOSPITAL_ID = 270001 ORDER BY USER_ID) AS t1 INNER JOIN (SELECT USER_ID AS CUS_ID, PROVINCE, CITY, BIRTHDAY, GENDER, REGISTER_DATE FROM qyw.qyw_4th_user ORDER BY CUS_ID) AS t2 ON t1.USER_ID = t2.CUS_ID) AS t3 WHERE PROVINCE IS NOT null AND CITY IS NOT null GROUP BY PROVINCE, CITY ORDER BY USER_CNT DESC;
    ''', con=conn)

In [ ]:
province_city_cnt = pd.Series(df['USER_CNT'].values, index=df['PROVINCE']+' '+df['CITY'], name='用户地域分布')

In [ ]:
fig = plt.figure(dpi=100, figsize=(8,8))
# first axes
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.8])
patches, texts, autotexts = ax1.pie(province_city_cnt[1:11], labels=province_city_cnt[1:11].index, autopct='%1.1f%%', colors=['yellowgreen', 'gold', 'lightskyblue', 'lightcoral'])
plt.setp(autotexts, fontproperties=myFont, size=tipSize)
plt.setp(texts, fontproperties=myFont, size=tipSize)
ax1.set_title(u'用户地域分布（第二位至第十一位）',fontproperties=myFont, size=titleSize)
ax1.set_aspect(1)
plt.show()

In [ ]:
df_all = pd.read_sql('''
SELECT HOSPITAL_ID, PROVINCE, CITY, COUNT(DISTINCT USER_ID) AS USER_CNT FROM (SELECT * FROM (SELECT * FROM qyw.qyw_4th_visit WHERE USER_ID > 0 ORDER BY USER_ID) AS t1 INNER JOIN (SELECT USER_ID AS CUS_ID, PROVINCE, CITY, BIRTHDAY, GENDER, REGISTER_DATE FROM qyw.qyw_4th_user ORDER BY CUS_ID) AS t2 ON t1.USER_ID = t2.CUS_ID) AS t3 WHERE PROVINCE IS NOT null AND CITY IS NOT null GROUP BY HOSPITAL_ID, PROVINCE, CITY ORDER BY HOSPITAL_ID, USER_CNT DESC;
''', con=conn)

In [ ]:
pd.DataFrame(df_all.groupby(['PROVINCE', 'CITY'])['USER_CNT'].sum()).reset_index().sort_values(['USER_CNT'],0,[False])

In [ ]:
df_new = df_all.groupby(['HOSPITAL_ID','PROVINCE'])['USER_CNT'].sum().reset_index().sort_values(['PROVINCE','HOSPITAL_ID'],0,[True, False])
df_new = pd.merge(df_new[df_new['PROVINCE']!=u'安徽'], df_new[df_new['PROVINCE']!=u'湖北'])
# useless below
# array_province = df_new['PROVINCE'].drop_duplicates().values
# hospitals = [{270001:u'武汉市中心医院'}, {5510002:u'安徽省中医院'}]
# hos_cnts = {270001:[], 5510002:[]}
# for hospital in hospitals:
#     for hid, hname in hospital.items():
#         for province in array_province:
#             if df_new[df_new['HOSPITAL_ID']==hid]['PROVINCE'] == None:
#                 hos_cnts[hid].append(0)
#             else:
#                 hos_cnts[hid].append(df_new[df_new['HOSPITAL_ID']==hid]['PROVINCE'])
# print hos_cnts
# useless above
df_new_pivot = df_new.pivot('PROVINCE', 'HOSPITAL_ID', 'USER_CNT') # equal to above
df_new_pivot = df_new_pivot.fillna(0)

import numpy as np
import matplotlib.pyplot as plt


n_groups = df_new_pivot.index.size

hid_270001 = df_new_pivot[270001]

hid_5510002 = df_new_pivot[5510002]

fig, ax = plt.subplots(dpi=100, figsize=(14,8))

index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, hid_270001, bar_width,
                 alpha=opacity,
                 color='b',
                 error_kw=error_config,
                 label=u'武汉市中心医院')
    
rects2 = plt.bar(index + bar_width, hid_5510002, bar_width,
                 alpha=opacity,
                 color='r',
                 error_kw=error_config,
                 label=u'安徽省中医院')

plt.xlabel(u'省份', fontproperties=myFont, size=titleSize)
plt.ylabel(u'用户数量', fontproperties=myFont, size=titleSize)
plt.title(u'用户地域分布', fontproperties=myFont, size=titleSize)
plt.xticks(index + bar_width, df_new_pivot.index, fontproperties=myFont, size=tipSize)
plt.legend(prop=font_manager.FontProperties(fname='/Library/Fonts/Songti.ttc', size=tipSize))

def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.005*height,
                '%d' % int(height),
                ha='center', va='bottom', fontproperties=myFont, size=tipSize)

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

In [3]:
df_mhs = pd.read_sql('''
    SELECT CONCAT_WS(':',SUBSTRING_INDEX(SUBSTRING_INDEX(VISIT_TIME,' ',-1),':',2),'00') AS VISIT_TIME, COUNT(*) AS OP_CNT FROM qyw.qyw_4th_visit GROUP BY SUBSTRING_INDEX(SUBSTRING_INDEX(VISIT_TIME,' ',-1),':',2) ORDER BY VISIT_TIME;
    ''', con=conn)

In [4]:
df_mhs_series = pd.Series(df_mhs['OP_CNT'].values/7, index=df_mhs['VISIT_TIME'], name=u'日均操作时间分布')
df_mhs_series.index.name = u'操作时间'
ax = df_mhs_series.plot(figsize=(14,8))
for label in ax.get_xticklabels(): #xtick
    label.set_fontproperties(myFont)
for label in ax.get_yticklabels(): #ytick
    label.set_fontproperties(myFont)
for label in ax.get_label(): # legend
    label.set_fontproperties(myFont)
ax.set_title(df_mhs_series.name, fontproperties=myFont, size=titleSize)
ax.set_xlabel(df_mhs_series.index.name, fontproperties=myFont, size=tipSize)
ax.set_ylabel(u'操作数量', fontproperties=myFont, size=tipSize)
plt.show()

In [5]:
'a' in 'abc'

True

In [ ]:
conn.close()